Importing all the libraries I will be needing in the project :

In [2]:
pip install tweepy

     |████████████████████████████████| 94 kB 1.6 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
import requests
import json
import os
from bs4 import BeautifulSoup
import tweepy
import io
import warnings
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime

In [5]:
#hiding all the warnings I might come across later on
warnings.filterwarnings('ignore')

# Gather
Importing the twitter_archives csv file which I downloaded manually, into a pandas dataframe :

In [6]:
twitter_archives = pd.read_csv('twitter-archive-enhanced.csv')

Dowloading the image predictions file programmatically :

In [7]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)

Writing this url content to a file on memory :

In [8]:
with open(url.split('/')[-1], 'wb') as file:
    file.write(response.content)

Reading in a tsv file containing the breed predictions derived from all the dog images in the tweets :

In [9]:
image_predictions = pd.read_csv('image-predictions.tsv', sep = '\t')

In [10]:
consumer_key = 'YOUR CONSUMER KEY'
consumer_secret = 'YOUR CONSUMER SECRET'
access_token = 'YOUR ACCESS TOKEN'
access_secret = 'YOUR ACCESS SECRET'

Set up an application where I got my consumer key and secret and on connecting it to my twitter account I got my access token and secret as well.

Creating an api object by using the tweepy.API class, to query the twitter API :

In [11]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

In [13]:
api = tweepy.API(auth, wait_on_rate_limit = True)

Creating a list of all the tweet ids in the twitter archives dataset : 

In [14]:
tweet_ids_list = list(twitter_archives['tweet_id'])

Getting access to the content of all the tweets by using tweepy's get_status method on their respective ids and writing the json data of all those tweets to a text file with each tweet's json data on its own line :

In [15]:
del_tweet_ids = []
start_time = datetime.now()
with open('tweet_json.txt', 'a') as f:
    for tweet_id in tweet_ids_list:
        try:
            tweet = api.get_status(tweet_id, tweet_mode = 'extended')
            json.dump( tweet._json, f)
            f.write('\n')
            print(tweet_id)
        except:
            del_tweet_ids.append(tweet_id)
            print(tweet_id)
time_elapsed = datetime.now() - start_time
print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))

892420643555336193
892177421306343426
891815181378084864
891689557279858688
891327558926688256
891087950875897856
890971913173991426
890729181411237888
890609185150312448
890240255349198849
890006608113172480
889880896479866881
889665388333682689
889638837579907072
889531135344209921
889278841981685760
888917238123831296
888804989199671297
888554962724278272
888202515573088257
888078434458587136
887705289381826560
887517139158093824
887473957103951883
887343217045368832
887101392804085760
886983233522544640
886736880519319552
886680336477933568
886366144734445568
886267009285017600
886258384151887873
886054160059072513
885984800019947520
885528943205470208
885518971528720385
885311592912609280
885167619883638784
884925521741709313
884876753390489601
884562892145688576
884441805382717440
884247878851493888
884162670584377345
883838122936631299
883482846933004288
883360690899218434
883117836046086144
882992080364220416
882762694511734784
882627270321602560
882268110199369728
882045870035

In [16]:
del_tweet_ids

[892420643555336193,
 892177421306343426,
 891815181378084864,
 891689557279858688,
 891327558926688256,
 891087950875897856,
 890971913173991426,
 890729181411237888,
 890609185150312448,
 890240255349198849,
 890006608113172480,
 889880896479866881,
 889665388333682689,
 889638837579907072,
 889531135344209921,
 889278841981685760,
 888917238123831296,
 888804989199671297,
 888554962724278272,
 888202515573088257,
 888078434458587136,
 887705289381826560,
 887517139158093824,
 887473957103951883,
 887343217045368832,
 887101392804085760,
 886983233522544640,
 886736880519319552,
 886680336477933568,
 886366144734445568,
 886267009285017600,
 886258384151887873,
 886054160059072513,
 885984800019947520,
 885528943205470208,
 885518971528720385,
 885311592912609280,
 885167619883638784,
 884925521741709313,
 884876753390489601,
 884562892145688576,
 884441805382717440,
 884247878851493888,
 884162670584377345,
 883838122936631299,
 883482846933004288,
 883360690899218434,
 883117836046

Now that I know the tweet ids of all the tweets that don't exist anymore, I will put them in a list for further use :

In [17]:
del_tweet_ids = ['888202515573088257', '873697596434513921', '869988702071779329', '866816280283807744', '861769973181624320',
                 '845459076796616705', '842892208864923648', '837012587749474308', '827228250799742977', '802247111496568832',
                 '775096608509886464']

Reading the text file I just created, line by line into an empty list. Also, since I am using the readline method here, each tweet's json is read into string form.

In [18]:
tweets_list = []
with open('tweet_json.txt') as f:
    for tweet_id in tweet_ids_list:
        if tweet_id not in del_tweet_ids:
            tweets_list.append(f.readline()) 

On checking out the tweets_list I found out that there were a few empty strings in there, so I removed them by using some list comprehension :

In [19]:
tweets_list_updated = [i for i in tweets_list if i != '']

Creating a list of dictionaries with each tweet's id, retweet count and favourite count. Since each tweet's json is in string form, I need to convert them first by using the json.loads method.

In [20]:
count_list = []
for e in tweets_list_updated:
    tweet_id = json.loads(e)['id']
    retweet_count = json.loads(e)['retweet_count']
    favourite_count = json.loads(e)['favorite_count']
    count_list.append({'tweet_id': tweet_id, 'retweet_count': retweet_count, 'favourite_count': favourite_count})

In [21]:
tweet_counts = pd.DataFrame(count_list, columns = ['tweet_id', 'retweet_count', 'favourite_count'])

In [22]:
tweet_counts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2345 entries, 0 to 2344
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   tweet_id         2345 non-null   int64
 1   retweet_count    2345 non-null   int64
 2   favourite_count  2345 non-null   int64
dtypes: int64(3)
memory usage: 55.1 KB


So on checking out the dataframe, 2345 tweet ids remain out of the 2356 ids which were originally there in the twitter archives dataset. Now that I have gathered all the datasets I need, I will have to assess each one of them to resolve some of the quality and tidiness issues in them. A clean and properly structured dataset is vital for conducting good analysis.
# Assess

I will start with the twitter archives dataframe imported earlier.

In [23]:
#increasing the column width so that the whole text in the 'text' column is visible.
pd.set_option('display.max_colwidth', -1)

In [24]:
twitter_archives.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",12,10,Franklin,None,None,None,None


On using my first function only, I can see that a lot of columns here have NaN values.

In [25]:
twitter_archives.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

There are a lot of columns here having more null values than non-null values and I don't even think if these columns will be of any use when it comes to conducting analysis or visualisations. Features like in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id aren't of much use here since all of them act as unique identifiers and we already have the tweet id column as the primary key here. This is actually a tidiness issue since these columns aren't really needed.

Also, tweet_id should be a string instead of an integer since I want it to be a primary key and anyways we won't be doing any calculations with it. The data type for the timestamp column should be datetime instead of string.  

In [26]:
#filter out all the rows where retweeted_status_id is null
twitter_archives[- twitter_archives.retweeted_status_id.isnull()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
19,888202515573088257,NaN,NaN,2017-07-21 01:02:36 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @dog_rates: This is Canela. She attempted some fancy porch pics. They were unsuccessful. 13/10 someone help her https://t.co/cLyzpcUcMX,8.874740e+17,4.196984e+09,2017-07-19 00:47:34 +0000,"https://twitter.com/dog_rates/status/887473957103951883/photo/1,https://twitter.com/dog_rates/status/887473957103951883/photo/1,https://twitter.com/dog_rates/status/887473957103951883/photo/1,https://twitter.com/dog_rates/status/887473957103951883/photo/1",13,10,Canela,None,None,None,None
32,886054160059072513,NaN,NaN,2017-07-15 02:45:48 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @Athletics: 12/10 #BATP https://t.co/WxwJmvjfxo,8.860537e+17,1.960740e+07,2017-07-15 02:44:07 +0000,"https://twitter.com/dog_rates/status/886053434075471873,https://twitter.com/dog_rates/status/886053434075471873",12,10,None,None,None,None,None
36,885311592912609280,NaN,NaN,2017-07-13 01:35:06 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @dog_rates: This is Lilly. She just parallel barked. Kindly requests a reward now. 13/10 would pet so well https://t.co/SATN4If5H5,8.305833e+17,4.196984e+09,2017-02-12 01:04:29 +0000,"https://twitter.com/dog_rates/status/830583320585068544/photo/1,https://twitter.com/dog_rates/status/830583320585068544/photo/1,https://twitter.com/dog_rates/status/830583320585068544/photo/1,https://twitter.com/dog_rates/status/830583320585068544/photo/1",13,10,Lilly,None,None,None,None
68,879130579576475649,NaN,NaN,2017-06-26 00:13:58 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @dog_rates: This is Emmy. She was adopted today. Massive round of pupplause for Emmy and her new family. 14/10 for all involved https://…,8.780576e+17,4.196984e+09,2017-06-23 01:10:23 +0000,"https://twitter.com/dog_rates/status/878057613040115712/photo/1,https://twitter.com/dog_rates/status/878057613040115712/photo/1",14,10,Emmy,None,None,None,None
73,878404777348136964,NaN,NaN,2017-06-24 00:09:53 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","RT @dog_rates: Meet Shadow. In an attempt to reach maximum zooming borkdrive, he tore his ACL. Still 13/10 tho. Help him out below\n\nhttps:/…",8.782815e+17,4.196984e+09,2017-06-23 16:00:04 +0000,"https://www.gofundme.com/3yd6y1c,https://twitter.com/dog_rates/status/878281511006478336/photo/1",13,10,Shadow,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1023,746521445350707200,NaN,NaN,2016-06-25 01:52:36 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @dog_rates: This is Shaggy. He knows exactly how to solve the puzzle but can't talk. All he wants to do is help. 10/10 great guy https:/…,6.678667e+17,4.196984e+09,2015-11-21 00:46:50 +0000,https://twitter.com/dog_rates/status/667866724293877760/photo/1,10,10,Shaggy,None,None,None,None
1043,743835915802583040,NaN,NaN,2016-06-17 16:01:16 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @dog_rates: Extremely intelligent dog here. Has learned to walk like human. Even has his own dog. Very impressive 10/10 https://t.co/0Dv…,6.671383e+17,4.196984e+09,2015-11-19 00:32:12 +0000,https://twitter.com/dog_rates/status/667138269671505920/photo/1,10,10,None,None,None,None,None
1242,711998809858043904,NaN,NaN,2016-03-21 19:31:59 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @twitter: @dog_rates Awesome Tweet! 12/10. Would Retweet. #LoveT

This table lists out all the rows with retweets. These aren't really valid as they are duplicated tweets and we don't want retweets. 

In [27]:
twitter_archives.tail()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2351,666049248165822465,NaN,NaN,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here we have a 1949 1st generation vulpix. Enjoys sweat tea and Fox News. Cannot be phased. 5/10 https://t.co/4B7cOc1EDq,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666049248165822465/photo/1,5,10,None,None,None,None,None
2352,666044226329800704,NaN,NaN,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a purebred Piers Morgan. Loves to Netflix and chill. Always looks like he forgot to unplug the iron. 6/10 https://t.co/DWnyCjf2mx,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666044226329800704/photo/1,6,10,a,None,None,None,None
2353,666033412701032449,NaN,NaN,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here is a very happy pup. Big fan of well-maintained decks. Just look at that tongue. 9/10 would cuddle af https://t.co/y671yMhoiR,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666033412701032449/photo/1,9,10,a,None,None,None,None
2354,666029285002620928,NaN,NaN,2015-11-15 23:05:30 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a western brown Mitsubishi terrier. Upset about leaf. Actually 2 dogs here. 7/10 would walk the shit out of https://t.co/r7mOb2m0UI,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666029285002620928/photo/1,7,10,a,None,None,None,None
2355,666020888022790149,NaN,NaN,2015-11-15 22:32:08 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here we have a Japanese Irish Setter. Lost eye in Vietnam (?). Big fan of relaxing on stair. 8/10 would pet https://t.co/BLDqew2Ijj,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666020888022790149/photo/1,8,10,None,None,None,None,None


On exploring the tail I see that some dogs have been given a default name 'None'.

In [28]:
twitter_archives[twitter_archives.name.duplicated()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",When you watch your owner call another dog a good boy but then they turn back to you and say you're a great boy. 13/10 https://t.co/v0nONBcwxq,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/890729181411237888/photo/1,https://twitter.com/dog_rates/status/890729181411237888/photo/1",13,10,None,None,None,None,None
12,889665388333682689,NaN,NaN,2017-07-25 01:55:32 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here's a puppo that seems to be on the fence about something haha no but seriously someone help her. 13/10 https://t.co/BxvuXk0UCm,NaN,NaN,NaN,https://twitter.com/dog_rates/status/889665388333682689/photo/1,13,10,None,None,None,None,puppo
23,887473957103951883,NaN,NaN,2017-07-19 00:47:34 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Canela. She attempted some fancy porch pics. They were unsuccessful. 13/10 someone help her https://t.co/cLyzpcUcMX,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/887473957103951883/photo/1,https://twitter.com/dog_rates/status/887473957103951883/photo/1",13,10,Canela,None,None,None,None
24,887343217045368832,NaN,NaN,2017-07-18 16:08:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",You may not have known you needed to see this today. 13/10 please enjoy (IG: emmylouroo) https://t.co/WZqNqygEyV,NaN,NaN,NaN,https://twitter.com/dog_rates/status/887343217045368832/video/1,13,10,None,None,None,None,None
25,887101392804085760,NaN,NaN,2017-07-18 00:07:08 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This... is a Jubilant Antarctic House Bear. We only rate dogs. Please only send dogs. Thank you... 12/10 would suffocate in floof https://t.co/4Ad1jzJSdp,NaN,NaN,NaN,https://twitter.com/dog_rates/status/887101392804085760/photo/1,12,10,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2351,666049248165822465,NaN,NaN,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here we have a 1949 1st generation vulpix. Enjoys sweat tea and Fox News. Cannot be phased. 5/10 https://t.co/4B7cOc1EDq,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666049248165822465/photo/1,5,10,None,None,None,None,None
2352,666044226329800704,NaN,NaN,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a purebred Piers Morgan. Loves to Netflix and chill. Always looks like he forgot to unplug the iron. 6/10 https://t.co/DWnyCjf2mx,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666044226329800704/photo/1,6,10,a,None,None,None,None
2353,666033412701032449,NaN,NaN,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here is a very happy pup. Big fan of well-maintained decks. Just look at that tongue. 9/10 would cuddle af https://t.co/y671yMhoiR,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666033412701032449/photo/1,9,10,a,None,None,None,None
2354,666029285002620928,NaN,NaN,2015-11-15 23:05:30 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a western brown Mitsubishi terrier. Upset about leaf. Actually 2 dogs here. 7/10 would walk the shit out of https://t.co/r7mOb2m0UI,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666029285002620928/photo/1,7,10,a,None,None,None,None


On checking out the duplicated values in the name feature its evident that maybe a lot of the dogs didn't have a name or that data got lost and got replaced by the default 'None'. Also there are a lot of irrelevant values in that column like 'a', 'quite', 'the' which clearly shows that the wrong text was extracted from the text column. This issue should come under the accuracy dimension of quality issues.

Also, the url for the tweet with tweet id 885518971528720385 seems to be broken as the page cannot be found.

In [29]:
twitter_archives.name.value_counts()

None          745
a             55 
Charlie       12 
Cooper        11 
Lucy          11 
              .. 
Dex           1  
Ace           1  
Tayzie        1  
Grizzie       1  
Christoper    1  
Name: name, Length: 957, dtype: int64

This lists out the counts of all the names in the name column and we can clearly see that there are names like None, a, an which aren't the right names.   

In [30]:
twitter_archives.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2037,671561002136281088,NaN,NaN,2015-12-01 05:26:34 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is the best thing I've ever seen so spread it like wildfire &amp; maybe we'll find the genius who created it. 13/10 https://t.co/q6RsuOVYwU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/671561002136281088/photo/1,13,10,the,None,None,None,None
1413,698907974262222848,NaN,NaN,2016-02-14 16:33:40 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This dog is never sure if he's doing the right thing. 10/10 https://t.co/GXq43zFfBu,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/698907974262222848/photo/1,https://twitter.com/dog_rates/status/698907974262222848/photo/1,https://twitter.com/dog_rates/status/698907974262222848/photo/1",10,10,None,None,None,None,None
1702,680934982542561280,NaN,NaN,2015-12-27 02:15:25 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Say hello to Sadie. She's a Tortellini Sidewinder. Very jubilant pup. Seems loyal. Leaves on point. 10/10 petable af https://t.co/g2bTu4ayPl,NaN,NaN,NaN,https://twitter.com/dog_rates/status/680934982542561280/photo/1,10,10,Sadie,None,None,None,None
928,754747087846248448,NaN,NaN,2016-07-17 18:38:22 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Keith. He's pursuing a more 2D lifestyle. Idiosyncratic af. 12/10 follow your dreams Keith https://t.co/G9ufksBMlU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/754747087846248448/photo/1,12,10,Keith,None,None,None,None
1251,710997087345876993,NaN,NaN,2016-03-19 01:11:29 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Meet Milo and Amos. They are the best of pals. Both 12/10 would pet at the same time https://t.co/Mv37BHEyyD,NaN,NaN,NaN,https://twitter.com/dog_rates/status/710997087345876993/photo/1,12,10,Milo,None,None,None,None


Viewing a random sample here shows up a Nan value in the expanded urls column. There are Nan values in the columns doggo, floofer, pupper and puppo as well but they are represented by 'None', that's why they didn't show up in the info table I generated earlier.

In [31]:
#getting the counts for all the ratings in the rating_numerator column
twitter_archives.rating_numerator.value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7       55 
14      54 
5       37 
6       32 
3       19 
4       17 
2       9  
1       9  
75      2  
15      2  
420     2  
0       2  
80      1  
144     1  
17      1  
26      1  
20      1  
121     1  
143     1  
44      1  
60      1  
45      1  
50      1  
99      1  
204     1  
1776    1  
165     1  
666     1  
27      1  
182     1  
24      1  
960     1  
84      1  
88      1  
Name: rating_numerator, dtype: int64

This throws up a lot of the ratings which don't look like they are correct. Most of the ratings given by the twitter handle are 12, 11, 10, 13 and so on whereas only a couple of dogs or even less have been given a rating of 80, 420, 165, 960 and more of that order. These ratings seem to be accuracy errors. 

In [32]:
#the actual rating below should be 9.75 instead of 75
twitter_archives[twitter_archives.rating_numerator == 75]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
340,832215909146226688,NaN,NaN,2017-02-16 13:11:49 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","RT @dog_rates: This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wu…",7.867091e+17,4.196984e+09,2016-10-13 23:23:56 +0000,https://twitter.com/dog_rates/status/786709082849828864/photo/1,75,10,Logan,None,None,None,None
695,786709082849828864,NaN,NaN,2016-10-13 23:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS",NaN,NaN,NaN,https://twitter.com/dog_rates/status/786709082849828864/photo/1,75,10,Logan,None,None,None,None


Although on looking up the text for the tweets corresponding to these ratings, I found out that there were decimals in some of these ratings and the number before the decimal point has been wrongfully scraped off in some cases. Thus this also points out to the fact that ratings numerator should be of data type float.

In [33]:
##getting the counts for all the ratings in the rating_denominator column
twitter_archives.rating_denominator.value_counts()

10     2333
11     3   
50     3   
20     2   
80     2   
70     1   
7      1   
15     1   
150    1   
170    1   
0      1   
90     1   
40     1   
130    1   
110    1   
16     1   
120    1   
2      1   
Name: rating_denominator, dtype: int64

Again this shows that there can be inaccurate values in this column as well since the total rating is 10.

In [34]:
#filtering out all the rows where the name is None
twitter_archives[twitter_archives.name == 'None']

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here we have a majestic great white breaching off South Africa's coast. Absolutely h*ckin breathtaking. 13/10 (IG: tucker_marlo) #BarkWeek https://t.co/kQ04fDDRmh,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950875897856/photo/1,13,10,None,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",When you watch your owner call another dog a good boy but then they turn back to you and say you're a great boy. 13/10 https://t.co/v0nONBcwxq,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/890729181411237888/photo/1,https://twitter.com/dog_rates/status/890729181411237888/photo/1",13,10,None,None,None,None,None
12,889665388333682689,NaN,NaN,2017-07-25 01:55:32 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here's a puppo that seems to be on the fence about something haha no but seriously someone help her. 13/10 https://t.co/BxvuXk0UCm,NaN,NaN,NaN,https://twitter.com/dog_rates/status/889665388333682689/photo/1,13,10,None,None,None,None,puppo
24,887343217045368832,NaN,NaN,2017-07-18 16:08:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",You may not have known you needed to see this today. 13/10 please enjoy (IG: emmylouroo) https://t.co/WZqNqygEyV,NaN,NaN,NaN,https://twitter.com/dog_rates/status/887343217045368832/video/1,13,10,None,None,None,None,None
25,887101392804085760,NaN,NaN,2017-07-18 00:07:08 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This... is a Jubilant Antarctic House Bear. We only rate dogs. Please only send dogs. Thank you... 12/10 would suffocate in floof https://t.co/4Ad1jzJSdp,NaN,NaN,NaN,https://twitter.com/dog_rates/status/887101392804085760/photo/1,12,10,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2342,666082916733198337,NaN,NaN,2015-11-16 02:38:37 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here we have a well-established sunblockerspaniel. Lost his other flip-flop. 6/10 not very waterproof https://t.co/3RU6x0vHB7,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666082916733198337/photo/1,6,10,None,None,None,None,None
2343,666073100786774016,NaN,NaN,2015-11-16 01:59:36 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Let's hope this flight isn't Malaysian (lol). What a dog! Almost completely camouflaged. 10/10 I trust this pilot https://t.co/Yk6GHE9tOY,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666073100786774016/photo/1,10,10,None,None,None,None,None
2344,666071193221509120,NaN,NaN,2015-11-16 01:52:02 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here we have a northern speckled Rhododendron. Much sass. Gives 0 fucks. Good tongue. 9/10 would caress sensually https://t.co/ZoL8kq2XFx,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666071193221509120/photo/1,9,10,None,None,None,None,None
2351,666049248165822465,NaN,NaN,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here we have a 1949 1st generation vulpix. Enjoys sweat tea and Fox News. Cannot be phased. 5/10 https://t.co/4B7cOc1EDq,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666049248165822465/photo/1,5,10,None,None,None,None,None


On visually assessing the text column here in an Excel sheet, I found out that a lot of people were sending pictures of other animals or things to rate as well. So such tweets are invalid.

In [35]:
twitter_archives.source.value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                        91  
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                     33  
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>    11  
Name: source, dtype: int64

The source column, since it consists of urls, is a bit untidy with all the non-alphanumeric characters and thus a little difficult to read.

Let's move on to the image predictions table now.

In [36]:
image_predictions

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
...,...,...,...,...,...,...,...,...,...,...,...,...
2070,891327558926688256,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True
2071,891689557279858688,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False
2072,891815181378084864,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True
2073,892177421306343426,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True


On visually assessing this table, I can see that some of the predictions in the p1 column aren't breeds of dogs but other animals or non-living things and on checking out the corresponding image urls on a browser myself, I found out that a lot of these images didn't have any dogs in them. Thus such tweets are invalid. 

In [37]:
image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


Again I would prefer if tweet_id is a string rather than an integer. Everything else seems to be ok here.

In [38]:
image_predictions.tail(10)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
2065,890240255349198849,https://pbs.twimg.com/media/DFrEyVuW0AAO3t9.jpg,1,Pembroke,0.511319,True,Cardigan,0.451038,True,Chihuahua,0.029248,True
2066,890609185150312448,https://pbs.twimg.com/media/DFwUU__XcAEpyXI.jpg,1,Irish_terrier,0.487574,True,Irish_setter,0.193054,True,Chesapeake_Bay_retriever,0.118184,True
2067,890729181411237888,https://pbs.twimg.com/media/DFyBahAVwAAhUTd.jpg,2,Pomeranian,0.566142,True,Eskimo_dog,0.178406,True,Pembroke,0.076507,True
2068,890971913173991426,https://pbs.twimg.com/media/DF1eOmZXUAALUcq.jpg,1,Appenzeller,0.341703,True,Border_collie,0.199287,True,ice_lolly,0.193548,False
2069,891087950875897856,https://pbs.twimg.com/media/DF3HwyEWsAABqE6.jpg,1,Chesapeake_Bay_retriever,0.425595,True,Irish_terrier,0.116317,True,Indian_elephant,0.076902,False
2070,891327558926688256,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True
2071,891689557279858688,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False
2072,891815181378084864,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True
2073,892177421306343426,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True
2074,892420643555336193,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,1,orange,0.097049,False,bagel,0.085851,False,banana,0.076110,False


In [39]:
image_predictions.sample(4)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1101,720785406564900865,https://pbs.twimg.com/media/CgC-gMCWcAAawUE.jpg,1,Chihuahua,0.896422,True,dingo,0.027929,False,kelpie,0.017916,True
674,683391852557561860,https://pbs.twimg.com/media/CXvlQ2zW8AAE0tp.jpg,1,French_bulldog,0.992833,True,Boston_bull,0.004749,True,pug,0.001392,True
1260,748705597323898880,https://pbs.twimg.com/ext_tw_video_thumb/748704826305970176/pu/img/QHuadM5eEygfBeOf.jpg,1,tiger_shark,0.548497,False,great_white_shark,0.130252,False,scuba_diver,0.121887,False
251,670704688707301377,https://pbs.twimg.com/media/CU7SW39WwAAL8Rw.jpg,1,Norwich_terrier,0.419838,True,cairn,0.351876,True,Norfolk_terrier,0.051094,True


The underscores visible in the predictions columns make the data look a little untidy and underprepared.

In [40]:
#getting the counts of all the prediction values in the p1 column
image_predictions.p1.value_counts()

golden_retriever      150
Labrador_retriever    100
Pembroke              89 
Chihuahua             83 
pug                   57 
                      .. 
pillow                1  
carousel              1  
bald_eagle            1  
lorikeet              1  
orange                1  
Name: p1, Length: 378, dtype: int64

The top predictions for a lot of the tweets have been other animals or things as well as the list above shows. 

In [41]:
#filtering out all the rows where the value of p1 is 'web_site'
image_predictions[image_predictions.p1 == 'web_site']

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
94,667550882905632768,https://pbs.twimg.com/media/CUObvUJVEAAnYPF.jpg,1,web_site,0.998258,False,dishwasher,0.000201,False,oscilloscope,0.000142,False
95,667550904950915073,https://pbs.twimg.com/media/CUOb_gUUkAACXdS.jpg,1,web_site,0.999335,False,vizsla,0.000081,True,collie,0.000069,True
130,668291999406125056,https://pbs.twimg.com/media/CUZABzGW4AE5F0k.jpg,1,web_site,0.995535,False,skunk,0.001363,False,badger,0.000686,False
213,670040295598354432,https://pbs.twimg.com/media/CUx2F6lVEAAvFev.jpg,1,web_site,0.901552,False,borzoi,0.026660,True,Chihuahua,0.012438,True
270,670822709593571328,https://pbs.twimg.com/media/CU89schWIAIHQmA.jpg,1,web_site,0.993887,False,Chihuahua,0.001252,True,menu,0.000599,False
715,685547936038666240,https://pbs.twimg.com/media/CYOONfZW8AA7IOA.jpg,1,web_site,0.923987,False,oscilloscope,0.009712,False,hand-held_computer,0.008769,False
960,705786532653883392,https://pbs.twimg.com/media/Cct1G6vVAAI9ZjF.jpg,1,web_site,0.550294,False,Labrador_retriever,0.148496,True,golden_retriever,0.148482,True
1210,742465774154047488,https://pbs.twimg.com/media/Ck3EribXEAAPhZn.jpg,1,web_site,0.997154,False,comic_book,0.000439,False,desktop_computer,0.000268,False
1317,755206590534418437,https://pbs.twimg.com/media/CnsIT0WWcAAul8V.jpg,1,web_site,0.906673,False,printer,0.008600,False,carton,0.004533,False
1443,775729183532220416,https://pbs.twimg.com/media/CsPxk85XEAAeMQj.jpg,1,web_site,0.989407,False,hand-held_computer,0.002139,False,menu,0.002115,False


A lot of the images here are not necessarily dog pictures but snapshots or private chats as well. Also, there are some retweets here. The first two tweets in the table above are retweets. I found that out by cross checking in the twitter archives table below. 

In [42]:
twitter_archives[twitter_archives.tweet_id == 667550882905632768] 

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2260,667550882905632768,NaN,NaN,2015-11-20 03:51:47 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Twitter Web Client</a>",RT @dogratingrating: Unoriginal idea. Blatant plagiarism. Curious grammar. -5/10 https://t.co/r7XzeQZWzb,6.675484e+17,4.296832e+09,2015-11-20 03:41:59 +0000,"https://twitter.com/dogratingrating/status/667548415174144001/photo/1,https://twitter.com/dogratingrating/status/667548415174144001/photo/1",5,10,None,None,None,None,None


Let's check out the tweet_counts table now.

In [43]:
tweet_counts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2345 entries, 0 to 2344
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   tweet_id         2345 non-null   int64
 1   retweet_count    2345 non-null   int64
 2   favourite_count  2345 non-null   int64
dtypes: int64(3)
memory usage: 55.1 KB


The favourite count column here is of interest and I would like to join that column to the image predictions table.

In [44]:
#summarizing the columns in the table
tweet_counts.describe()

,tweet_id,retweet_count,favourite_count
count,2.345000e+03,2345.000000,2345.000000
mean,7.422940e+17,3050.126652,8111.325373
std,6.833642e+16,5070.651402,12197.466796
min,6.660209e+17,0.000000,0.000000
25%,6.783802e+17,610.000000,1408.000000
50%,7.189392e+17,1424.000000,3559.000000
75%,7.986979e+17,3557.000000,10026.000000
max,8.924206e+17,77908.000000,144098.000000


In [45]:
tweet_counts.head()

,tweet_id,retweet_count,favourite_count
0,892420643555336193,8662,39018
1,892177421306343426,6361,33407
2,891815181378084864,4224,25188
3,891689557279858688,8770,42374
4,891327558926688256,9538,40526


### Quality
#### Twitter archives table
- Most of the values in the in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id and retweeted_status_timestamp columns are null and these columns only act as identifiers.
- Missing records in the expanded_urls column (2297 out of 2356).
- Retweets in many rows.
- Incorrect data types for tweet_id, timestamp and rating_numerator.
- Inaccurate dog names in the name column and NaN values represented by the word 'None'.
- Inaccurate values in the rating_numerator and rating_denominator columns.
- The source column is a bit untidy cause of the urls and thus a little difficult to interpret.
- The tweet with the dog name Phineas has been duplicated.

#### Image predictions table
- Incorrect data type for tweet id.
- Underscores in place of spaces in the p1, p2, p3 prediction columns.
- The values in the columns p1_conf, p2_conf and p3_conf should be percentages instead of proportions.
- Inconsistent letter cases in the columns p1, p2 and p3. 

#### Counts table
- Incorrect data type for tweet id.

### Tidiness
#### Twitter archives table
- Doggo, floofer, pupper, puppo should be column values but are instead column headers.
- Two values in the timestamp column: date and time.
- Adding the favourite count column to this table from the tweet counts table.

#### Image predictions table
- Joining this table and the twitter archives table.

# Clean

In [46]:
#making copies of the three tables to conduct cleaning operations on
twitter_archives_clean = twitter_archives.copy()
image_predictions_clean = image_predictions.copy()
tweet_counts_clean = tweet_counts.copy()

## Twitter archives table
### Define
Remove all the retweets by removing all the rows with non null values in retweeted_status_id.

### Code

In [47]:
#subsetting the table over all the rows having null values in the retweeted_status_id column
twitter_archives_clean = twitter_archives_clean[twitter_archives_clean.retweeted_status_id.isnull()]

### Test

In [48]:
twitter_archives_clean[twitter_archives_clean.retweeted_status_id.notnull()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [49]:
twitter_archives_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2175 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2175 non-null   object 
 4   source                      2175 non-null   object 
 5   text                        2175 non-null   object 
 6   retweeted_status_id         0 non-null      float64
 7   retweeted_status_user_id    0 non-null      float64
 8   retweeted_status_timestamp  0 non-null      object 
 9   expanded_urls               2117 non-null   object 
 10  rating_numerator            2175 non-null   int64  
 11  rating_denominator          2175 non-null   int64  
 12  name                        2175 non-null   object 
 13  doggo                       2175 

### Define
Drop the in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id and retweeted_status_timestamp columns.

### Code

In [50]:
twitter_archives_clean = twitter_archives_clean.drop(['in_reply_to_status_id', 'in_reply_to_user_id',
                                                      'retweeted_status_id', 'retweeted_status_user_id',
                                                      'retweeted_status_timestamp'], axis = 1)

### Test

In [51]:
twitter_archives_clean.head(3)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,None,None,None,None
1,892177421306343426,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,None,None,None,None
2,891815181378084864,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,None,None,None,None


### Define
Create a new column named doy_type with doggo, floofer, pupper, puppo as its values; extracted from the text column and drop the doggo, floofer, pupper, puppo columns. Also, change the datatype of the new column created to category.

### Code

In [52]:
#creating the column dog_type
twitter_archives_clean['dog_type'] = twitter_archives_clean.text.str.extract('(doggo | floofer | pupper | puppo)', expand = True)

#some whitespaces also got extracted to the dog_type column, thus removing them
twitter_archives_clean.dog_type = twitter_archives_clean.dog_type.str.strip()

In [53]:
#dropping the columns
twitter_archives_clean = twitter_archives_clean.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis = 1)

In [54]:
#changing the datatype of dog_type to category
twitter_archives_clean.dog_type = twitter_archives_clean.dog_type.astype('category')

### Test

In [55]:
twitter_archives_clean

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,dog_type
0,892420643555336193,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,NaN
1,892177421306343426,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,NaN
2,891815181378084864,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,NaN
3,891689557279858688,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13,10,Darla,NaN
4,891327558926688256,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f","https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",12,10,Franklin,NaN
...,...,...,...,...,...,...,...,...,...
2351,666049248165822465,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here we have a 1949 1st generation vulpix. Enjoys sweat tea and Fox News. Cannot be phased. 5/10 https://t.co/4B7cOc1EDq,https://twitter.com/dog_rates/status/666049248165822465/photo/1,5,10,None,NaN
2352,666044226329800704,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a purebred Piers Morgan. Loves to Netflix and chill. Always looks like he forgot to unplug the iron. 6/10 https://t.co/DWnyCjf2mx,https://twitter.com/dog_rates/status/666044226329800704/photo/1,6,10,a,NaN
2353,666033412701032449,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here is a very happy pup. Big fan of well-maintained decks. Just look at that tongue. 9/10 would cuddle af https://t.co/y671yMhoiR,https://twitter.com/dog_rates/status/666033412701032449/photo/1,9,10,a,NaN
2354,666029285002620928,2015-11-15 23:05:30 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a western brown Mitsubishi terrier. Upset about leaf. Actually 2 dogs here. 7/10 would walk the shit out of https://t.co/r7mOb2m0UI,https://twitter.com/dog_rates/status/666029285002620928/photo/1,7,10,a,NaN


In [56]:
twitter_archives_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   tweet_id            2175 non-null   int64   
 1   timestamp           2175 non-null   object  
 2   source              2175 non-null   object  
 3   text                2175 non-null   object  
 4   expanded_urls       2117 non-null   object  
 5   rating_numerator    2175 non-null   int64   
 6   rating_denominator  2175 non-null   int64   
 7   name                2175 non-null   object  
 8   dog_type            240 non-null    category
dtypes: category(1), int64(3), object(5)
memory usage: 155.3+ KB


### Define
Do an inner join on this table and the image_predictions table since we only want tweets with images and no retweets. Also, drop the unnecessary columns after that. This will also remove all the rows with null values in the expanded_urls column since those are the tweets with no images and thus aren't there in the image predictions table.

### Code

In [57]:
twitter_archives_clean = twitter_archives_clean.merge(image_predictions_clean, on = 'tweet_id', how = 'inner')

In [58]:
#dropping the unnecessary columns
twitter_archives_clean = twitter_archives_clean.drop(['img_num', 'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog'], axis = 1)

### Test

In [59]:
twitter_archives_clean.head(4)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,dog_type,jpg_url,p1,p1_conf,p1_dog
0,892420643555336193,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,NaN,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,orange,0.097049,False
1,892177421306343426,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,NaN,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,Chihuahua,0.323581,True
2,891815181378084864,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,NaN,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,Chihuahua,0.716012,True
3,891689557279858688,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13,10,Darla,NaN,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,paper_towel,0.170278,False


### Define
Join the tweet_counts table to the twitter archives table to get the favourite and retweet counts.

### Clean

In [60]:
twitter_archives_clean = pd.merge(twitter_archives_clean, tweet_counts_clean, on = 'tweet_id', how = 'left')

### Test

In [61]:
twitter_archives_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 0 to 1993
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   tweet_id            1994 non-null   int64   
 1   timestamp           1994 non-null   object  
 2   source              1994 non-null   object  
 3   text                1994 non-null   object  
 4   expanded_urls       1994 non-null   object  
 5   rating_numerator    1994 non-null   int64   
 6   rating_denominator  1994 non-null   int64   
 7   name                1994 non-null   object  
 8   dog_type            216 non-null    category
 9   jpg_url             1994 non-null   object  
 10  p1                  1994 non-null   object  
 11  p1_conf             1994 non-null   float64 
 12  p1_dog              1994 non-null   bool    
 13  retweet_count       1994 non-null   int64   
 14  favourite_count     1994 non-null   int64   
dtypes: bool(1), category(1), float64(1), i

In [62]:
twitter_archives_clean.sample(3)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,dog_type,jpg_url,p1,p1_conf,p1_dog,retweet_count,favourite_count
1084,700796979434098688,2016-02-19 21:39:54 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Daniel. He's a neat pup. Exotic af. Custom paws. Leaps unannounced. Would totally pet. 7/10 daaamn Daniel https://t.co/5XaR0kj8cr,https://twitter.com/dog_rates/status/700796979434098688/photo/1,7,10,Daniel,NaN,https://pbs.twimg.com/media/Cbm7IeUXIAA6Lc-.jpg,tailed_frog,0.652712,False,1058,2623
1579,674019345211760640,2015-12-08 00:15:09 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Acro. You briefly see her out of the corner of your eye. You look and she's not there. 10/10 mysterious pup https://t.co/fqiEsTduEs,https://twitter.com/dog_rates/status/674019345211760640/photo/1,10,10,Acro,NaN,https://pbs.twimg.com/media/CVqZBO8WUAAd931.jpg,collie,0.992732,True,329,1186
1630,672898206762672129,2015-12-04 22:00:08 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Cheryl AKA Queen Pupper of the Skies. Experienced fighter pilot. Much skill. True hero. 11/10 https://t.co/i4XJEWwdsp,https://twitter.com/dog_rates/status/672898206762672129/photo/1,11,10,Cheryl,NaN,https://pbs.twimg.com/media/CVadWcCXIAAL4Sh.jpg,motor_scooter,0.835819,False,447,927


### Define
- Change the datatypes of the tweet_id, timestamp columns to string and datetime respectively.
- Change the datatype of the rating_numerator and rating_denominator columns to float.

### Code

In [63]:
twitter_archives_clean.tweet_id = twitter_archives_clean.tweet_id.astype(str)
twitter_archives_clean.timestamp = pd.to_datetime(twitter_archives_clean.timestamp, yearfirst = True)
twitter_archives_clean.rating_numerator = twitter_archives_clean.rating_numerator.astype(float)
twitter_archives_clean.rating_denominator = twitter_archives_clean.rating_numerator.astype(float)

### Test

In [64]:
twitter_archives_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 0 to 1993
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   tweet_id            1994 non-null   object             
 1   timestamp           1994 non-null   datetime64[ns, UTC]
 2   source              1994 non-null   object             
 3   text                1994 non-null   object             
 4   expanded_urls       1994 non-null   object             
 5   rating_numerator    1994 non-null   float64            
 6   rating_denominator  1994 non-null   float64            
 7   name                1994 non-null   object             
 8   dog_type            216 non-null    category           
 9   jpg_url             1994 non-null   object             
 10  p1                  1994 non-null   object             
 11  p1_conf             1994 non-null   float64            
 12  p1_dog              1994 non-null 

### Define
Extract the date and time from the timestamp column and put them into two new columns. Drop the timestamp column after that.

### Code

In [65]:
#using the apply function, applying the strftime function to each value of the timestamp column in each row
twitter_archives_clean['date'] = twitter_archives_clean['timestamp'].apply(lambda x: x.strftime('%d-%m-%Y'))
twitter_archives_clean['time'] = twitter_archives_clean['timestamp'].apply(lambda x: x.strftime('%H:%M:%S'))

#changing datatype of the date column to datetime
twitter_archives_clean.date = pd.to_datetime(twitter_archives_clean.date, dayfirst = True)

In [66]:
#dropping the timestamp column
twitter_archives_clean = twitter_archives_clean.drop('timestamp', axis = 1)

### Test

In [67]:
twitter_archives_clean.head(4)

,tweet_id,source,text,expanded_urls,rating_numerator,rating_denominator,name,dog_type,jpg_url,p1,p1_conf,p1_dog,retweet_count,favourite_count,date,time
0,892420643555336193,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13.0,13.0,Phineas,NaN,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,orange,0.097049,False,8662,39018,2017-08-01,16:23:56
1,892177421306343426,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",https://twitter.com/dog_rates/status/892177421306343426/photo/1,13.0,13.0,Tilly,NaN,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,Chihuahua,0.323581,True,6361,33407,2017-08-01,00:17:27
2,891815181378084864,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12.0,12.0,Archie,NaN,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,Chihuahua,0.716012,True,4224,25188,2017-07-31,00:18:03
3,891689557279858688,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13.0,13.0,Darla,NaN,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,paper_towel,0.170278,False,8770,42374,2017-07-30,15:58:51


### Define 
Make the source column readable by replacing the urls in it by the source name and change its datatype to category.

### Code

In [68]:
#the source column has only 3 different values, thus assigning each value to a different variable  
a = '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>'
b = '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>'
c = '<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>'

In [69]:
#using the variables created above in the replace function 
twitter_archives_clean.source = twitter_archives_clean.source.replace(a, 'Twitter for iPhone')
twitter_archives_clean.source = twitter_archives_clean.source.replace(b, 'Twitter Web Client')
twitter_archives_clean.source = twitter_archives_clean.source.replace(c, 'TweetDeck')

### Test

In [70]:
twitter_archives_clean.source.value_counts()

Twitter for iPhone    1955
Twitter Web Client    28  
TweetDeck             11  
Name: source, dtype: int64

### Define
- Rename some of the columns in the table and re-arrange them.
- Convert the proportions in the p1 confidence column into percentages. 
- Replace the underscores in the p1 column by spaces.
- Capitalize the first letter of every word in the p1 column.

### Code

In [71]:
#renaming some of the columns
twitter_archives_clean = twitter_archives_clean.rename({'jpg_url': 'image_url', 'p1':'prediction',
                                                        'p1_conf': 'confidence_percentage', 'p1_dog': 'breed_predicted'},
                                                       axis = 'columns')

In [72]:
#re-arranging all the columns for better viewing
twitter_archives_clean = twitter_archives_clean[['tweet_id', 'source', 'image_url', 'date', 'time', 'favourite_count',
                                                 'retweet_count', 'text', 'name', 'dog_type', 'rating_numerator',
                                                 'rating_denominator', 'prediction', 'confidence_percentage', 
                                                 'breed_predicted', 'expanded_urls']]

In [73]:
#changing the values in confidence level from proportions to percentages
twitter_archives_clean.confidence_percentage = twitter_archives_clean.confidence_percentage.apply(lambda x: round(x*100, 2))

In [74]:
#replacing the underscores in the prediction column by spaces
twitter_archives_clean.prediction = twitter_archives_clean.prediction.str.replace('_', ' ')

In [75]:
#changing the first letter of every word in the prediction column to uppercase
twitter_archives_clean.prediction = twitter_archives_clean.prediction.str.title()

### Test

In [76]:
twitter_archives_clean.head(3)

,tweet_id,source,image_url,date,time,favourite_count,retweet_count,text,name,dog_type,rating_numerator,rating_denominator,prediction,confidence_percentage,breed_predicted,expanded_urls
0,892420643555336193,Twitter for iPhone,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,2017-08-01,16:23:56,39018,8662,This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,Phineas,NaN,13.0,13.0,Orange,9.70,False,https://twitter.com/dog_rates/status/892420643555336193/photo/1
1,892177421306343426,Twitter for iPhone,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,2017-08-01,00:17:27,33407,6361,"This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",Tilly,NaN,13.0,13.0,Chihuahua,32.36,True,https://twitter.com/dog_rates/status/892177421306343426/photo/1
2,891815181378084864,Twitter for iPhone,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,2017-07-31,00:18:03,25188,4224,This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,Archie,NaN,12.0,12.0,Chihuahua,71.60,True,https://twitter.com/dog_rates/status/891815181378084864/photo/1


### Define
Replace the incorrect dog names i.e. the names in lowercases with the correct ones by extracting them from their corresponding text. Also replace the remaining 'None' in the name column by NaN values. 

### Code

In [77]:
#filtering out rows where there is the word 'named' in the text and the name is in lowercase - these are probably the rows where 
#the names are incorrect and then creating a list out of their corresponding indices
index_list = twitter_archives_clean.loc[twitter_archives_clean.text.str.contains('named') & 
                                              twitter_archives_clean.name.str.islower()].index.tolist()
all_indices = twitter_archives_clean.index.tolist()
for e in all_indices:
    if e in index_list:
        for ele in list(range(len(index_list))):
            #creating a list out of all such text values that contain the word 'named' and the corresponding name value 
            #is in lowercase            
            text_list = (twitter_archives_clean.loc[twitter_archives_clean.text.str.contains('named') & 
                                                 twitter_archives_clean.name.str.islower()].text).tolist()
            #finding the index position in every text value in the list where 'named' occurs
            num = text_list[ele].find('named')
            #using this index position to extract a particular pattern of dog name out of the text and then assign it to the
            #corresponding name value
            x = twitter_archives_clean.loc[twitter_archives_clean.index == e, 'text'].str[num+6:].str.extract(r'([A-Z][a-z]+)',
                                                                                                          expand = True)[0]
            twitter_archives_clean.loc[twitter_archives_clean.index == e, 'name'] = x
            break

In [78]:
#this is the same code as above, only tinkering it a bit to filter out rows with 'named' in the text and 'None' in the 
#name column
index_list = twitter_archives_clean.loc[(twitter_archives_clean.name == 'None') & 
                                        (twitter_archives_clean.text.str.contains('named'))].index.tolist()
for e in all_indices:
    if e in index_list:
        for ele in list(range(len(index_list))):
            text_list = (twitter_archives_clean.loc[(twitter_archives_clean.name == 'None') & 
                                                 (twitter_archives_clean.text.str.contains('named'))].text).tolist()
            num = text_list[ele].find('named')
            x = twitter_archives_clean.loc[twitter_archives_clean.index == e, 'text'].str[num+6:].str.extract(r'([A-Z][a-z]+)',
                                                                                                          expand = True)[0]
            twitter_archives_clean.loc[twitter_archives_clean.index == e, 'name'] = x
            break

In [79]:
#replacing the remaining 'None's in the name column by NaN values
twitter_archives_clean.name = twitter_archives_clean.name.replace('None', np.nan)

### Test

In [80]:
twitter_archives_clean.loc[twitter_archives_clean.text.str.contains('named')]

,tweet_id,source,image_url,date,time,favourite_count,retweet_count,text,name,dog_type,rating_numerator,rating_denominator,prediction,confidence_percentage,breed_predicted,expanded_urls
1503,675706639471788032,Twitter for iPhone,https://pbs.twimg.com/media/CWCXj35VEAIFvtk.jpg,2015-12-12,15:59:51,681,104,This is a Sizzlin Menorah spaniel from Brooklyn named Wylie. Lovable eyes. Chiller as hell. 10/10 and I'm out.. poof https://t.co/7E0AiJXPmI,Wylie,NaN,10.0,10.0,English Springer,99.03,True,https://twitter.com/dog_rates/status/675706639471788032/photo/1
1600,673636718965334016,Twitter for iPhone,https://pbs.twimg.com/media/CVk9ApFWUAA-S1s.jpg,2015-12-06,22:54:44,1174,398,This is a Lofted Aphrodisiac Terrier named Kip. Big fan of bed n breakfasts. Fits perfectly. 10/10 would pet firmly https://t.co/gKlLpNzIl3,Kip,NaN,10.0,10.0,Wombat,88.03,False,https://twitter.com/dog_rates/status/673636718965334016/photo/1
1679,671743150407421952,Twitter for iPhone,https://pbs.twimg.com/media/CVKC1IfWIAAsQks.jpg,2015-12-01,17:30:22,769,241,This is a Tuscaloosa Alcatraz named Jacob (Yacōb). Loves to sit in swing. Stellar tongue. 11/10 look at his feet https://t.co/2IslQ8ZSc7,Jacob,NaN,11.0,11.0,Toy Poodle,41.94,True,https://twitter.com/dog_rates/status/671743150407421952/photo/1
1710,671147085991960577,Twitter for iPhone,https://pbs.twimg.com/media/CVBktzQXAAAPpUA.jpg,2015-11-30,02:01:49,699,246,This is a Helvetica Listerine named Rufus. This time Rufus will be ready for the UPS guy. He'll never expect it 9/10 https://t.co/34OhVhMkVr,Rufus,NaN,9.0,9.0,Yorkshire Terrier,46.72,True,https://twitter.com/dog_rates/status/671147085991960577/photo/1
1760,670427002554466305,Twitter for iPhone,https://pbs.twimg.com/media/CU3VzVwWwAAAsst.jpg,2015-11-28,02:20:27,538,174,This is a Deciduous Trimester mix named Spork. Only 1 ear works. No seat belt. Incredibly reckless. 9/10 still cute https://t.co/CtuJoLHiDo,Spork,NaN,9.0,9.0,Seat Belt,95.23,False,https://twitter.com/dog_rates/status/670427002554466305/photo/1
1769,670361874861563904,Twitter for iPhone,https://pbs.twimg.com/media/CU2akCQWsAIbaOV.jpg,2015-11-27,22:01:40,341,71,This is a Rich Mahogany Seltzer named Cherokee. Just got destroyed by a snowball. Isn't very happy about it. 9/10 https://t.co/98ZBi6o4dj,Cherokee,NaN,9.0,9.0,Platypus,97.41,False,https://twitter.com/dog_rates/status/670361874861563904/photo/1
1772,670303360680108032,Twitter for iPhone,https://pbs.twimg.com/media/CU1lWFaVAAAl0HG.jpg,2015-11-27,18:09:09,443,145,This is a Speckled Cauliflower Yosemite named Hemry. He's terrified of intruder dog. Not one bit comfortable. 9/10 https://t.co/yV3Qgjh8iN,Hemry,NaN,9.0,9.0,Shetland Sheepdog,38.03,True,https://twitter.com/dog_rates/status/670303360680108032/photo/1
1790,669923323644657664,Twitter for iPhone,https://pbs.twimg.com/media/CUwLtPeU8AAfAb2.jpg,2015-11-26,16:59:01,245,62,This is a spotted Lipitor Rumpelstiltskin named Alphred. He can't wait for the Turkey. 10/10 would pet really well https://t.co/6GUGO7azNX,Alphred,NaN,10.0,10.0,Car Mirror,34.31,False,https://twitter.com/dog_rates/status/669923323644657664/photo/1
1804,669564461267722241,Twitter for iPhone,https://pbs.twimg.com/media/CUrFUvDVAAA9H-F.jpg,2015-11-25,17:13:02,404,131,This is a Coriander Baton Rouge named Alfredo. Loves to cuddle with smaller well-dressed dog. 10/10 would hug lots https://t.co/eCRdwouKCl,Alfredo,NaN,10.0,10.0,Toy Poodle,62.37,True,https://twitter.com/dog_rates/status/669564461267722241/photo/1
1809,669363888236994561,Twitter for iPhone,https://pbs.twimg.com/media/CUoO1TLWsAA0Z3w.jpg,2015-11-25,03:56:01,651,246,Here we have a Gingivitis Pumpernickel named Zeus. Unmatched tennis ball capacity. 10/10 would highly recommend https://t.co/jPkd7hhX7m,Zeus,NaN,10.0,10.0,Golden Retriever,53.90,True,https://twitter.com/dog_rates/status/669363888236994561/photo/1


In [81]:
twitter_archives_clean[twitter_archives_clean.name == 'None']

,tweet_id,source,image_url,date,time,favourite_count,retweet_count,text,name,dog_type,rating_numerator,rating_denominator,prediction,confidence_percentage,breed_predicted,expanded_urls


### Define
Replace the inaccurate values in the rating_numerator column, where the numbers before the decimal were scraped off, with the correct values in the corresponding text column.

### Code

In [82]:
#from all text values containing floating point numbers followed by a forward slash (/), extracting those numbers 
values = twitter_archives_clean.loc[twitter_archives_clean.text.str.contains('(\d+\.\d+\/)'), 'text'].str.extract('(\d+\.\d+)')
#at rows which contains the aforementioned pattern in the the text values, assigning the numbers extracted above to the
#rating_numerator column
twitter_archives_clean.loc[twitter_archives_clean.text.str.contains('(\d+\.\d+\/)'), 'rating_numerator'] = values

In [83]:
#on visually assessing I found out that the ratings at index 925 were incorrect, rating_numerator and rating_denominator 
#should be 11 and 10 respectively instead of 50
twitter_archives_clean.loc[twitter_archives_clean.index == 925, 'rating_numerator'] = 11
twitter_archives_clean.loc[twitter_archives_clean.index == 925, 'rating_denominator'] = 10

### Test

In [84]:
#filtering out the rows which contain text values with floating point numbers as ratings
twitter_archives_clean.loc[twitter_archives_clean.text.str.contains('(\d+\.\d+\/)')]

,tweet_id,source,image_url,date,time,favourite_count,retweet_count,text,name,dog_type,rating_numerator,rating_denominator,prediction,confidence_percentage,breed_predicted,expanded_urls
39,883482846933004288,Twitter for iPhone,https://pbs.twimg.com/media/DELC9dZXUAADqUk.jpg,2017-07-08,00:28:19,46235,10140,"This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948",Bella,NaN,NaN,5.0,Golden Retriever,94.31,True,"https://twitter.com/dog_rates/status/883482846933004288/photo/1,https://twitter.com/dog_rates/status/883482846933004288/photo/1"
503,786709082849828864,Twitter for iPhone,https://pbs.twimg.com/media/CurzvFTXgAA2_AP.jpg,2016-10-13,23:23:56,19980,6911,"This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS",Logan,NaN,NaN,75.0,Pomeranian,46.73,True,https://twitter.com/dog_rates/status/786709082849828864/photo/1
553,778027034220126208,Twitter for iPhone,https://pbs.twimg.com/media/Cswbc2yWcAAVsCJ.jpg,2016-09-20,00:24:34,7197,1837,This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back https://t.co/QFaUiIHxHq,Sophie,NaN,NaN,27.0,Clumber,94.67,True,https://twitter.com/dog_rates/status/778027034220126208/photo/1
1374,680494726643068929,Twitter for iPhone,https://pbs.twimg.com/media/CXGaVxOWAAADjhF.jpg,2015-12-25,21:06:00,1841,532,Here we have uncovered an entire battalion of holiday puppers. Average of 11.26/10 https://t.co/eNm2S6p9BD,NaN,NaN,NaN,26.0,Kuvasz,43.86,True,https://twitter.com/dog_rates/status/680494726643068929/photo/1


In [85]:
twitter_archives_clean[twitter_archives_clean.index == 925]

,tweet_id,source,image_url,date,time,favourite_count,retweet_count,text,name,dog_type,rating_numerator,rating_denominator,prediction,confidence_percentage,breed_predicted,expanded_urls
925,716439118184652801,Twitter for iPhone,https://pbs.twimg.com/media/CfFNk7cWAAA-hND.jpg,2016-04-03,01:36:11,2529,241,This is Bluebert. He just saw that both #FinalFur match ups are split 50/50. Amazed af. 11/10 https://t.co/Kky1DPG4iq,Bluebert,NaN,11.0,10.0,Siberian Husky,39.65,True,https://twitter.com/dog_rates/status/716439118184652801/photo/1


On visually assessing again I luckily found out that there are two tweets with the same dog name Phineas, image and almost similar texts.

In [86]:
twitter_archives_clean[twitter_archives_clean.name == 'Phineas']

,tweet_id,source,image_url,date,time,favourite_count,retweet_count,text,name,dog_type,rating_numerator,rating_denominator,prediction,confidence_percentage,breed_predicted,expanded_urls
0,892420643555336193,Twitter for iPhone,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,2017-08-01,16:23:56,39018,8662,This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,Phineas,NaN,13.0,13.0,Orange,9.70,False,https://twitter.com/dog_rates/status/892420643555336193/photo/1
1748,670668383499735048,Twitter for iPhone,https://pbs.twimg.com/media/CU6xVkbWsAAeHeU.jpg,2015-11-28,18:19:37,11227,5385,This is Phineas. He's a magical dog. Only appears through the hole of a donut. 10/10 mysterious pup https://t.co/NECxEHN5YU,Phineas,NaN,10.0,10.0,Banana,10.73,False,https://twitter.com/dog_rates/status/670668383499735048/photo/1


Thus removing the more recent tweet at index 0 :

In [87]:
twitter_archives_clean = twitter_archives_clean.drop(0, axis = 0)

In [88]:
#checking if the row was removed
twitter_archives_clean[twitter_archives_clean.name == 'Phineas']

,tweet_id,source,image_url,date,time,favourite_count,retweet_count,text,name,dog_type,rating_numerator,rating_denominator,prediction,confidence_percentage,breed_predicted,expanded_urls
1748,670668383499735048,Twitter for iPhone,https://pbs.twimg.com/media/CU6xVkbWsAAeHeU.jpg,2015-11-28,18:19:37,11227,5385,This is Phineas. He's a magical dog. Only appears through the hole of a donut. 10/10 mysterious pup https://t.co/NECxEHN5YU,Phineas,NaN,10.0,10.0,Banana,10.73,False,https://twitter.com/dog_rates/status/670668383499735048/photo/1


## Image predictions table
### Define
Change tweet_id column's datatype from int to object.

### Code

In [89]:
image_predictions_clean.tweet_id = image_predictions_clean.tweet_id.astype(str)

### Test

In [90]:
image_predictions_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   object 
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(1), object(5)
memory usage: 152.1+ KB


### Define
Replace the underscores in the p1, p2 and p3 columns by spaces.

### Code

In [91]:
#replacing using the replace function
image_predictions_clean.p1 = image_predictions_clean.p1.str.replace('_',' ')
image_predictions_clean.p2 = image_predictions_clean.p2.str.replace('_',' ')
image_predictions_clean.p3 = image_predictions_clean.p3.str.replace('_',' ')

### Test

In [92]:
image_predictions_clean[['p1', 'p2', 'p3']]

,p1,p2,p3
0,Welsh springer spaniel,collie,Shetland sheepdog
1,redbone,miniature pinscher,Rhodesian ridgeback
2,German shepherd,malinois,bloodhound
3,Rhodesian ridgeback,redbone,miniature pinscher
4,miniature pinscher,Rottweiler,Doberman
...,...,...,...
2070,basset,English springer,German short-haired pointer
2071,paper towel,Labrador retriever,spatula
2072,Chihuahua,malamute,kelpie
2073,Chihuahua,Pekinese,papillon


### Define
Convert the proportions in the p1_conf, p2_conf and p3_conf columns into percentages.

### Code

In [93]:
#using apply, multiplying 100 to each column value in each row
image_predictions_clean.p1_conf = image_predictions_clean.p1_conf.apply(lambda x: round(x*100, 2))
image_predictions_clean.p2_conf = image_predictions_clean.p2_conf.apply(lambda x: round(x*100, 2))
image_predictions_clean.p3_conf = image_predictions_clean.p3_conf.apply(lambda x: round(x*100, 2))

### Test

In [94]:
image_predictions_clean

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh springer spaniel,46.51,True,collie,15.67,True,Shetland sheepdog,6.14,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,50.68,True,miniature pinscher,7.42,True,Rhodesian ridgeback,7.20,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German shepherd,59.65,True,malinois,13.86,True,bloodhound,11.62,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian ridgeback,40.81,True,redbone,36.07,True,miniature pinscher,22.28,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature pinscher,56.03,True,Rottweiler,24.37,True,Doberman,15.46,True
...,...,...,...,...,...,...,...,...,...,...,...,...
2070,891327558926688256,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,basset,55.57,True,English springer,22.58,True,German short-haired pointer,17.52,True
2071,891689557279858688,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper towel,17.03,False,Labrador retriever,16.81,True,spatula,4.08,False
2072,891815181378084864,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,71.60,True,malamute,7.83,True,kelpie,3.14,True
2073,892177421306343426,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1,Chihuahua,32.36,True,Pekinese,9.06,True,papillon,6.90,True


### Define
Capitalize the first letter of every word in the columns p1, p2 and p3.

### Code

In [95]:
#capitalizing by using the str.title function
image_predictions_clean.p1 = image_predictions_clean.p1.str.title()
image_predictions_clean.p2 = image_predictions_clean.p2.str.title()
image_predictions_clean.p3 = image_predictions_clean.p3.str.title()

### Test

In [96]:
image_predictions_clean.head(10)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh Springer Spaniel,46.51,True,Collie,15.67,True,Shetland Sheepdog,6.14,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,Redbone,50.68,True,Miniature Pinscher,7.42,True,Rhodesian Ridgeback,7.20,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German Shepherd,59.65,True,Malinois,13.86,True,Bloodhound,11.62,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian Ridgeback,40.81,True,Redbone,36.07,True,Miniature Pinscher,22.28,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,Miniature Pinscher,56.03,True,Rottweiler,24.37,True,Doberman,15.46,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese Mountain Dog,65.11,True,English Springer,26.38,True,Greater Swiss Mountain Dog,1.62,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,Box Turtle,93.30,False,Mud Turtle,4.59,False,Terrapin,1.79,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,Chow,69.25,True,Tibetan Mastiff,5.83,True,Fur Coat,5.44,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,Shopping Cart,96.25,False,Shopping Basket,1.46,False,Golden Retriever,0.80,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,Miniature Poodle,20.15,True,Komondor,19.23,True,Soft-Coated Wheaten Terrier,8.21,True


### Define
Change the column names of jpg_url, p1, p1_conf, p1_dog, p2, p2_conf, p2_dog, p3, p3_conf, p3_dog for better viewing.

### Code

In [97]:
image_predictions_clean = image_predictions_clean.rename({'jpg_url': 'image_url', 'p1': 'prediction_1',
                                                          'p1_conf': 'confidence_percent_1', 'p1_dog': 'breed_predicted_1',
                                                          'p2': 'prediction_2', 'p2_conf': 'confidence_percent_2',
                                                          'p2_dog': 'breed_predicted_2', 'p3': 'prediction_3',
                                                          'p3_conf': 'confidence_percent_3', 'p3_dog': 'breed_predicted_3'},
                                                         axis = 'columns')

### Test

In [98]:
image_predictions_clean.head()

,tweet_id,image_url,img_num,prediction_1,confidence_percent_1,breed_predicted_1,prediction_2,confidence_percent_2,breed_predicted_2,prediction_3,confidence_percent_3,breed_predicted_3
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh Springer Spaniel,46.51,True,Collie,15.67,True,Shetland Sheepdog,6.14,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,Redbone,50.68,True,Miniature Pinscher,7.42,True,Rhodesian Ridgeback,7.20,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German Shepherd,59.65,True,Malinois,13.86,True,Bloodhound,11.62,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian Ridgeback,40.81,True,Redbone,36.07,True,Miniature Pinscher,22.28,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,Miniature Pinscher,56.03,True,Rottweiler,24.37,True,Doberman,15.46,True


## Tweet counts table
### Define
Change tweet_id column's datatype from int to object.

### Code

In [99]:
tweet_counts_clean.tweet_id = tweet_counts_clean.tweet_id.astype(str)

### Test

In [100]:
tweet_counts_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2345 entries, 0 to 2344
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   tweet_id         2345 non-null   object
 1   retweet_count    2345 non-null   int64 
 2   favourite_count  2345 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 55.1+ KB


## Storing
Storing the final cleaned datasets to csv files.

In [101]:
twitter_archives_clean.to_csv('twitter_archive_master.csv', index = False)
image_predictions_clean.to_csv('image_predicitons_clean.csv', index = False)
tweet_counts_clean.to_csv('tweet_counts_clean.csv', index = False)

## Analysis and Visualization

In [102]:
#reading the cleaned twitter archives csv file for analysis
twitter_data = pd.read_csv('twitter_archive_master.csv', encoding = 'utf-8')

In [103]:
twitter_data

,tweet_id,source,image_url,date,time,favourite_count,retweet_count,text,name,dog_type,rating_numerator,rating_denominator,prediction,confidence_percentage,breed_predicted,expanded_urls
0,892177421306343426,Twitter for iPhone,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,2017-08-01,00:17:27,33407,6361,"This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",Tilly,NaN,13.0,13.0,Chihuahua,32.36,True,https://twitter.com/dog_rates/status/892177421306343426/photo/1
1,891815181378084864,Twitter for iPhone,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,2017-07-31,00:18:03,25188,4224,This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,Archie,NaN,12.0,12.0,Chihuahua,71.60,True,https://twitter.com/dog_rates/status/891815181378084864/photo/1
2,891689557279858688,Twitter for iPhone,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,2017-07-30,15:58:51,42374,8770,This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,Darla,NaN,13.0,13.0,Paper Towel,17.03,False,https://twitter.com/dog_rates/status/891689557279858688/photo/1
3,891327558926688256,Twitter for iPhone,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2017-07-29,16:00:24,40526,9538,"This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f",Franklin,NaN,12.0,12.0,Basset,55.57,True,"https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1"
4,891087950875897856,Twitter for iPhone,https://pbs.twimg.com/media/DF3HwyEWsAABqE6.jpg,2017-07-29,00:08:17,20331,3166,Here we have a majestic great white breaching off South Africa's coast. Absolutely h*ckin breathtaking. 13/10 (IG: tucker_marlo) #BarkWeek https://t.co/kQ04fDDRmh,NaN,NaN,13.0,13.0,Chesapeake Bay Retriever,42.56,True,https://twitter.com/dog_rates/status/891087950875897856/photo/1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1988,666049248165822465,Twitter for iPhone,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,2015-11-16,00:24:50,109,40,Here we have a 1949 1st generation vulpix. Enjoys sweat tea and Fox News. Cannot be phased. 5/10 https://t.co/4B7cOc1EDq,NaN,NaN,5.0,5.0,Miniature Pinscher,56.03,True,https://twitter.com/dog_rates/status/666049248165822465/photo/1
1989,666044226329800704,Twitter for iPhone,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,2015-11-16,00:04:52,302,141,This is a purebred Piers Morgan. Loves to Netflix and chill. Always looks like he forgot to unplug the iron. 6/10 https://t.co/DWnyCjf2mx,a,NaN,6.0,6.0,Rhodesian Ridgeback,40.81,True,https://twitter.com/dog_rates/status/666044226329800704/photo/1
1990,666033412701032449,Twitter for iPhone,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,2015-11-15,23:21:54,125,45,Here is a very happy pup. Big fan of well-maintained decks. Just look at that tongue. 9/10 would cuddle af https://t.co/y671yMhoiR,a,NaN,9.0,9.0,German Shepherd,59.65,True,https://twitter.com/dog_rates/status/666033412701032449/photo/1
1991,666029285002620928,Twitter for iPhone,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,2015-11-15,23:05:30,131,47,This is a western brown Mitsubishi terrier. Upset about leaf. Actually 2 dogs here. 7/10 would walk the shit out of https://t.co/r7mOb2m0UI,a,NaN,7.0,7.0,Redbone,50.68,True,https://twitter.com/dog_rates/status/666029285002620928/photo/1


For the purpose of analysis I will divide this dataset into two parts: one with all the rows where dog breeds were predicted and the other where no dog breeds were predicted.

In [104]:
only_breeds = twitter_data[twitter_data.breed_predicted == True]
no_breeds = twitter_data[twitter_data.breed_predicted == False]

In [105]:
only_breeds.prediction.value_counts().head(10)

Golden Retriever      139
Labrador Retriever    95 
Pembroke              88 
Chihuahua             79 
Pug                   54 
Chow                  41 
Samoyed               40 
Toy Poodle            38 
Pomeranian            38 
Malamute              29 
Name: prediction, dtype: int64

These are the top 10 most common dog breeds predicted out of all the dog pictures that were sent in.

In [106]:
only_breeds.groupby('prediction')['favourite_count'].mean().sort_values(ascending = False).head(10)

prediction
Saluki                     23760.250000
French Bulldog             18417.692308
Afghan Hound               17034.333333
Black-And-Tan Coonhound    16749.500000
Flat-Coated Retriever      16554.375000
Irish Water Spaniel        16132.000000
Standard Poodle            15564.000000
English Springer           15441.444444
Cardigan                   15072.823529
Leonberg                   14679.666667
Name: favourite_count, dtype: float64

Here are the top 10 breeds predicted with the highest mean favourite counts. None of the top most common dog breeds of this dataset make the cut here. Do these same breeds have the highest mean retweet counts? Let's check that out.

In [107]:
only_breeds.groupby('prediction')['retweet_count'].mean().sort_values(ascending = False).head(10)

prediction
Standard Poodle     6487.857143
English Springer    5855.777778
Afghan Hound        5834.666667
Saluki              5024.250000
French Bulldog      4751.961538
Eskimo Dog          4659.722222
Great Pyrenees      4613.571429
Lakeland Terrier    4526.875000
Samoyed             4512.750000
Cardigan            4436.411765
Name: retweet_count, dtype: float64

Though not in the same order but this list is almost the same as the one before. Only Eskimo dog, Great Pyrenees, Lakeland Terrrier and the Samoyed don't make it to the favourite counts list.

In [108]:
twitter_data.groupby('breed_predicted')['confidence_percentage'].mean().sort_values(ascending = False)

breed_predicted
True     61.436513
False    53.644690
Name: confidence_percentage, dtype: float64

These are the mean confidence levels for predictions that were dog breeds and predictions that weren't. It seems the neural network used for prediction was more confident when the prediction was a dog breed.

In [109]:
mean_ratings = only_breeds.groupby('prediction')['rating_numerator', 'favourite_count', 'retweet_count'].mean()
mean_ratings.sort_values(by = 'rating_numerator', ascending = False).head(10)

,rating_numerator,favourite_count,retweet_count
prediction,,,
Soft-Coated Wheaten Terrier,25.454545,2243.272727,796.181818
West Highland White Terrier,15.642857,6408.785714,1520.642857
Great Pyrenees,14.928571,12715.285714,4613.571429
Borzoi,14.444444,6115.777778,1894.222222
Labrador Retriever,13.905263,11644.473684,3774.378947
Golden Retriever,13.268116,12164.302158,3573.338129
Saluki,12.500000,23760.250000,5024.250000
Briard,12.333333,9358.666667,2898.333333
Tibetan Mastiff,12.250000,11980.000000,4087.250000


This table shows the top 10 breeds with the highest mean ratings and their corresponding mean favourite and retweet counts. The favourite and retweet counts are actually lower as compared to that of other breeds. 

In [110]:
twitter_data.groupby(['dog_type'])['rating_numerator'].mean().sort_values(ascending = False)

dog_type
floofer    12.500000
puppo      12.111111
doggo      11.695652
pupper     10.780142
Name: rating_numerator, dtype: float64

This table shows the mean rating for every dog type category. It seems like floofers and puppos were highly rated dogs.

In [111]:
only_breeds.groupby(['dog_type'])['prediction'].value_counts()

dog_type  prediction          
doggo     Golden Retriever        5
          Labrador Retriever      4
          German Shepherd         3
          Cocker Spaniel          2
          Dalmatian               2
                                 ..
puppo     Miniature Pinscher      1
          Old English Sheepdog    1
          Pembroke                1
          Rottweiler              1
          Siberian Husky          1
Name: prediction, Length: 91, dtype: int64

This list shows the value counts for every breed in each dog type category. Only two dogs were categorised as floofers and they were of the breeds Chow and Golden Retriever. The number of dogs classified as puppos also doesn't seem to be very high as compared to pupper and doggo.

In [112]:
twitter_data.groupby('dog_type')['favourite_count'].describe()

,count,mean,std,min,25%,50%,75%,max
dog_type,,,,,,,,
doggo,46.0,18264.195652,21683.919817,2548.0,6930.75,12524.0,19812.25,129032.0
floofer,2.0,17471.500000,7367.345553,12262.0,14866.75,17471.5,20076.25,22681.0
pupper,141.0,7167.170213,12868.766654,676.0,2365.00,3217.0,6616.00,124792.0
puppo,27.0,22720.888889,28078.104821,3227.0,7113.00,16074.0,23224.50,144098.0


This table displays all the descriptive statistics for favourite count in every dog type category. From this it looks like puppos were the most popular dogs (to no surprise since they are usually puppies). Their mean and max count values are the highest in their respective departments.

Let's check out the descriptive statistics of some of the features in the two datasets I created earlier.  

In [113]:
only_breeds.iloc[ : , 1:].describe()

,favourite_count,retweet_count,rating_numerator,rating_denominator,confidence_percentage
count,1477.000000,1477.000000,1473.000000,1477.000000,1477.000000
mean,9244.136087,2750.877454,11.479973,11.538253,61.436513
std,12910.754871,4782.806085,7.634734,7.822865,26.034166
min,80.000000,13.000000,1.000000,1.000000,4.430000
25%,2188.000000,643.000000,10.000000,10.000000,39.210000
50%,4431.000000,1418.000000,11.000000,11.000000,61.570000
75%,11570.000000,3183.000000,12.000000,12.000000,85.320000
max,144098.000000,77908.000000,165.000000,165.000000,100.000000


In [114]:
no_breeds.iloc[: , 1:].describe()

,favourite_count,retweet_count,rating_numerator,rating_denominator,confidence_percentage
count,516.000000,516.000000,516.000000,516.000000,516.000000
mean,7850.664729,2695.908915,14.327519,14.327519,53.644690
std,11665.736841,4565.961911,80.500204,80.500204,29.491312
min,105.000000,34.000000,0.000000,0.000000,5.900000
25%,1480.750000,568.750000,8.000000,8.000000,27.695000
50%,3116.000000,1104.000000,10.000000,10.000000,47.905000
75%,9303.250000,2814.000000,12.000000,12.000000,82.175000
max,92998.000000,39789.000000,1776.000000,1776.000000,100.000000


There are 1477 rows with breeds predicted in them to 516 with non-breeds. All the statistics seem to be greater in the breeds dataset except the numerator rating which is slightly higher in the non-breeds table.